# BMI Topography to GRASS GIS

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from bmi_topography import Topography
import subprocess
import sys

In [ ]:
# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

## Create Project

In [ ]:
!grass -e -c EPSG:32613 $HOME/bmi-topography-grass/colorado

## Start GRASS Session

In [ ]:
# Start GRASS Session
gj.init("./colorado/PERMANENT/")

In [ ]:
print(gs.read_command("g.region", flags="p"))

## Fetch Data with `bmi_topography`

In [ ]:
width = 0.05
dem = Topography(
    north=40.16 + width,
    south=40.14 - width,
    east=-105.4 + width,
    west=-105.5 - width,
    dem_type="SRTMGL3",
    output_format="GTiff"
)

fname = dem.fetch()

## Import DEM into GRASS

In [ ]:
# Import
gs.run_command("r.import", input=fname, output="dem", resolution="value", resolution_value=50)

In [ ]:
# Change computational region to DEM
gs.run_command("g.region", raster="dem", flags="p")

In [ ]:
omg = gj.Map()
omg.d_rast(map="dem")
omg.show()

## Insert your GRASS analysis here...

## Some Export Options

### Export to GeoTiff

In [ ]:
gs.run_command("r.out.gdal", input="dem", output="dem.tif")

Check your file browser panel and you should see a new file!

### Export to RasterGridModel

In [ ]:
# Additional Imports
import grass.script.array as garray
from landlab import RasterModelGrid, imshow_grid

We're going to use info about the raster to determine the spacing and shape of the RasterModelGrid. `raster_info` returns a dictionary where we can get the resolution and dimensions of the raster.

In [ ]:
gs.raster_info(map="dem")

In [ ]:
raster_info = gs.raster_info(map="dem")

# Make sure the east and west resolution is the same (since RasterModelGrid has one xy_spacing value)
assert raster_info['ewres']==raster_info['nsres']

# Get spacing and shape for RasterModelGrid
spacing = raster_info['nsres']
shape = (int(raster_info['rows']), int(raster_info['cols']))

Now that we have the spacing and shape of the array, we'll get the raster as an array and then flip and flatten to match the RasterModelGrid pattern.

In [ ]:
# Get GRASS Elevation Raster as np array
elev = garray.array("dem")

# Flip and flatten to match RasterModelGrid node pattern
flip = np.flip(elev, 0)
flat_elev = flip.flatten()

Finally, make the grid object and assign node values!

In [ ]:
# Make our RasterModelGrid using info from GRASS
grid = RasterModelGrid(shape, xy_spacing=spacing)
grid.at_node["topographic__elevation"] = flat_elev

Check out the results..

In [ ]:
imshow_grid(grid,'topographic__elevation')